https://www.youtube.com/watch?v=oWZQb8nRGfk&ab_channel=ArturSpirin

In [1]:
class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_column_info(self):
        
        column_info = []
        columns = self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[1]/thead/tr/th')
        for column in columns:
            column_info.append(str(column.text))
        return column_info
        
    def get_results(self, index=None):
        columns = self.get_column_info()
        data = {}
        elements = self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[1]/tbody/tr{}'
                                                      .format('[{}]'.format(index) if index else ''))
        
        for element in elements:
            current_index = elements.index(element) + 1 if not index else index #xpath counter starts at 1
            parsed_data = {}
            for column in columns:
                value = element.find_element_by_xpath('/html/body/div[2]/div[3]/div/table[1]/tbody/tr[{}]/td[{}]'
                                                      .format(current_index, columns.index(column)+1)).text
                parsed_data.update({column: str(value)})
            
            data.update({current_index: parsed_data})
            
        return data
    
    def get_number_of_results(self):
        
        return len(self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[1]/tbody/tr'))
    
    def click(self, column, value):
        
        pass

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
import time

In [3]:
start_year = 2000
end_year = 2021

driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')

for year in range(start_year, end_year):
    driver.get('https://basketball.realgm.com/nba/draft/past_drafts/{}'.format(str(year)))

    table = Table(driver)

    if year == start_year:
        df = pd.DataFrame(table.get_results()).T
        df['draft_year'] = year
        df = df.reset_index(drop=True)
    else:
        df_temp = pd.DataFrame(table.get_results()).T
        df_temp['draft_year'] = year
        df = pd.concat([df, df_temp])
        df = df.reset_index(drop=True)

df
#     print(table.get_column_info())
#     print(table.get_results())
#     print(table.get_number_of_results())

,Pick,Player,Team,Draft Trades,Pos,HT,WT,Age,YOS,Pre-Draft Team,Class,Nationality,draft_year,
0,1,Kenyon Martin,NJN,,F,6-9,240,22,15,Cincinnati,Sr,United States,2000,NaN
1,2,Stromile Swift,VAN,,F-C,6-9,225,20,9,LSU,So *,United States,2000,NaN
2,3,Darius Miles,LAC,,F,6-9,210,18,9,East St. Louis Senior High School (Illinois),1981 DOB *,United States,2000,NaN
3,4,Marcus Fizer,CHI,,F,6-8,246,21,6,Iowa State,Jr *,United States,2000,NaN
4,5,Mike Miller,ORL,,GF,6-8,218,20,17,Florida,So *,United States,2000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,26,Payton Pritchard,BOS,,PG,6-2,190,22,1,Oregon,Sr,United States,2020,NaN
619,27,Udoka Azubuike,UTH,,C,7-0,260,21,1,Kansas,Sr,Nigeria,2020,NaN
620,28,Jaden McDaniels,LAL,LAL to OKC\nOKC to MIN,F,6-10,185,20,1,Washington,Fr *,United States,2020,NaN
621,29,Malachi Flynn,TOR,,PG,6-1,185,22,1,San Diego State,Jr *,United States,2020,NaN


In [4]:
df.to_csv('C:/Users/17178/Documents/DS Projects/FanDuel/NBA Draft Project/Data/nba_draft_first_round.csv')